# NBA HOF Predictor
## Data Manipulation Process

In [39]:
import pandas as pd

## Hall of Fame Dataset

In [45]:
player = pd.read_csv("nba-hof.csv")

player.loc[player["hof"] == True].sort_values("num_seasons")

,player_id,player,birth_year,hof,num_seasons,first_seas,last_seas
826,827,John Thompson,NaN,True,2,1965,1966
166,167,Buddy Jeannette,1917.0,True,3,1948,1950
601,602,Maurice Stokes,NaN,True,3,1956,1958
452,453,Don Barksdale,NaN,True,4,1952,1955
2500,2501,Dražen Petrović,NaN,True,4,1990,1993
...,...,...,...,...,...,...,...
2999,3000,Kobe Bryant,NaN,True,20,1997,2016
1722,1723,Robert Parish,NaN,True,21,1977,1997
1604,1605,Moses Malone,1955.0,True,21,1975,1995
3115,3116,Dirk Nowitzki,NaN,True,21,1999,2019


## Player Accolades

In [ ]:
all_star = pd.read_csv("data/nba-all-star.csv")
awards = pd.read_csv("data/nba-awards.csv")
end_season = pd.read_csv("data/nba-end-season-teams.csv")
longevity = pd.read_csv("data/nba-longevity.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/nba-all-star.csv'

In [42]:
all_star_count = all_star.groupby(["player"]).count().sort_values("season", ascending=False)["team"].reset_index(
    name="num_all_star"
)
all_star_count

NameError: name 'all_star' is not defined

In [41]:
# Giving All Star players Player IDs
as_df = pd.merge(
    all_star_count,
    player,
    on = ["player"],
    how="inner"
)[["player_id", "player", "num_all_star", "num_seasons"]]

NameError: name 'all_star_count' is not defined

In [ ]:
award_count = awards.loc[awards["winner"] == True].groupby(
    ["player_id", "player", "award"]
).count()["season"].reset_index(name = "award_count")

award_count_wide = award_count.pivot_table(
    index=["player_id", "player"],
    columns="award",
    values="award_count",
    fill_value=0
).reset_index()

In [ ]:
end_season_award = end_season.groupby(["player_id", "player", "type", "number_tm"]).count()["season"].reset_index(
    name = "all_team_count")

end_season_award["team_award"] = end_season_award["type"] + " " + end_season_award["number_tm"]

esa_wide = end_season_award.pivot_table(
    index=["player_id", "player"],
    columns="team_award",
    values="all_team_count",
    fill_value=0
).reset_index()

In [36]:
# Combining all accolade tables

merge1 = pd.merge(
    as_df,
    award_count_wide,
    on=["player_id", "player"],
    how="inner"
)

accolades = pd.merge(
    merge1,
    esa_wide,
    on=["player_id", "player"],
    how="inner"
)

NameError: name 'as_df' is not defined

## Player Stats

In [4]:
averages = pd.read_csv("nba-averages.csv")
totals = pd.read_csv("nba-totals.csv")

In [29]:
a = averages.loc[(averages["lg"] == "NBA") | (averages["lg"] == "ABA")].drop(
    columns=["seas_id", "birth_year", "pos", "age", "experience", "tm", "lg", "season"]).fillna(0).groupby(
        ["player_id", "player"]
    ).mean().reset_index()

a

,player_id,player,g,gs,mp_per_game,fg_per_game,fga_per_game,fg_percent,x3p_per_game,x3pa_per_game,...,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game
0,0.0,Kenyon Martin Jr.,68.666667,19.666667,24.233333,4.033333,7.50,0.5370,0.8,2.266667,...,0.6760,1.366667,3.5,4.9,1.30,0.533333,0.6,0.9,1.733333,10.266667
1,0.0,Lester Quinones,4.000000,0.000000,4.500000,0.500000,1.30,0.4000,0.5,1.000000,...,0.6670,0.300000,0.5,0.8,0.50,0.300000,0.0,1.3,0.000000,2.500000
2,0.0,Phillip Wheeler,3.000000,0.000000,11.300000,1.000000,5.30,0.1880,0.0,2.300000,...,1.0000,0.700000,1.3,2.0,0.70,0.300000,0.7,0.3,0.000000,2.700000
3,0.0,Tyson Etienne,1.000000,0.000000,10.000000,3.000000,7.00,0.4290,2.0,5.000000,...,0.0000,0.000000,0.0,0.0,1.00,0.000000,0.0,0.0,2.000000,8.000000
4,5.0,Ariel Maughan,51.500000,0.000000,0.000000,2.300000,7.75,0.2955,0.0,0.000000,...,0.7815,0.000000,0.0,2.0,1.45,0.000000,0.0,0.0,2.600000,7.150000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117,5307.0,Yuri Collins,2.000000,0.000000,8.000000,0.500000,1.50,0.3330,0.0,0.000000,...,0.0000,1.000000,0.5,1.5,2.00,1.000000,0.0,0.0,2.500000,1.000000
5118,5308.0,Yves Missi,69.000000,63.000000,26.900000,3.600000,6.60,0.5430,0.0,0.000000,...,0.6340,3.500000,4.7,8.1,1.30,0.500000,1.3,1.2,2.000000,9.000000
5119,5309.0,Zaccharie Risacher,67.000000,65.000000,24.800000,4.700000,10.30,0.4520,1.6,4.600000,...,0.7170,1.200000,2.4,3.7,1.20,0.700000,0.5,1.3,1.900000,12.400000
5120,5310.0,Zach Edey,60.000000,49.000000,20.500000,3.800000,6.60,0.5750,0.3,0.800000,...,0.7220,3.500000,4.1,7.6,0.90,0.500000,1.3,1.2,2.700000,9.200000


In [33]:
t = totals.loc[(averages["lg"] == "NBA") | (averages["lg"] == "ABA")].drop(
    columns=["seas_id", "birth_year", "pos", "age", "experience", "tm", "lg", "season", "fg_percent", "g", "gs", "ft_percent"]).fillna(0).groupby(
        ["player_id", "player"]
    ).sum().reset_index()

t

,player_id,player,mp,fg,fga,x3p,x3pa,x3p_percent,x2p,x2pa,...,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,0.0,Kenyon Martin Jr.,5016.0,851,1561,163.0,480.0,1.037,688,1081,...,429,279.0,712.0,991.0,272,105.0,111.0,189.0,349,2154
1,0.0,Lester Quinones,18.0,2,5,2.0,4.0,0.500,0,1,...,6,1.0,2.0,3.0,2,1.0,0.0,5.0,0,10
2,0.0,Phillip Wheeler,34.0,3,16,0.0,7.0,0.000,3,9,...,2,2.0,4.0,6.0,2,1.0,2.0,1.0,0,8
3,0.0,Tyson Etienne,10.0,3,7,2.0,5.0,0.400,1,2,...,0,0.0,0.0,0.0,1,0.0,0.0,0.0,2,8
4,5.0,Ariel Maughan,0.0,238,824,0.0,0.0,0.000,238,824,...,328,0.0,0.0,141.0,149,0.0,0.0,0.0,265,731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117,5307.0,Yuri Collins,16.0,1,3,0.0,0.0,0.000,1,3,...,0,2.0,1.0,3.0,4,2.0,0.0,0.0,5,2
5118,5308.0,Yves Missi,1853.0,248,457,0.0,1.0,0.000,248,456,...,194,240.0,321.0,561.0,90,35.0,93.0,80.0,139,619
5119,5309.0,Zaccharie Risacher,1660.0,313,692,107.0,306.0,0.350,206,386,...,138,82.0,164.0,246.0,83,49.0,32.0,85.0,129,832
5120,5310.0,Zach Edey,1232.0,229,398,18.0,47.0,0.383,211,351,...,108,208.0,246.0,454.0,56,32.0,75.0,74.0,162,554


In [35]:
stats = pd.merge(
    t,
    a,
    on = ["player_id", "player"],
    how="inner"
)

pd.merge(
    accolades,
    stats,
    on = ["player_id", "player"],
    how = "inner"
)

NameError: name 'accolades' is not defined